# Crise du sida Berlin
Ce notebook Jupiter contient un **[tableau](#Tableau)** de tous les lieux visités pendant l'atelier topographique sur la crise du sida à Berlin dans les années 80 (*appel à participation [DE](https://www.sciencespo.fr/histoire/fr/node/89436.html) | [FR](https://cmb.hu-berlin.de/fr/agenda/evenement/topografischer-geschichtsworkshop-fuer-junge-forscherinnen)*), et une **[carte dynamique](#Carte)** générée à partir de ce tableau. 

L'atelier s'est tenu au Marc Bloch Zentrum du 13 au 18 mai 2024 et a entre autres donné lieu à un interview de l'une des organisatrices se trouve [ici](https://www.sciencespo.fr/fr/actualites/la-crise-du-sida-a-berlin-parcourir-une-ville-pour-comprendre-son-histoire/), la création d'une [librairie Zotero](https://www.zotero.org/groups/5537257/aidskrisebln2024/library) et d'un [répertoire de travail commun](https://sharedocs.huma-num.fr/wl/?id=Su3ebB9UxlLW3jueknVkCwTB4B0ckciH), afin d'archiver ce travail sur l'archive et de le poursuivre plus tard. 

## Tableau 

In [9]:
import pandas as pd
import folium

df_raw = pd.read_csv("synth-lieux.csv", encoding='utf-8', sep=';')
df = df_raw[df_raw['Visited'] > 30]
df

,Place,Adresse,Type,Timeline,Community,Keywords,Accessibility,East or west,Date
0,Centre Marc Bloch,"52.5107745,13.3896707",Research Center,1992,"Researchers, queer activists","Academic reflexivity, community, long-term tra...",NaN,East,13.05
1,Chez Michel,"52.5025988,13.4199707",Restaurant,1980,French,Taking care of primary body needs,NaN,West,13.05
2,Yorkstrasse,"52.4916585,13.3660462",Train station,"1981, 1903, 1939","Lesbians, gays, queers","Meeting, circulation, community, togetherness,...",NaN,West,14.05
3,Les Migras,"52.4919769,13.3646303",Community center,1999,"Lesbians, migrants","Counceling, advocating, community, emotional s...",NaN,West,14.05
4,Lesbenberatung,"52.4919769,13.3646303",Community center,1981,Lesbians,"Counceling, advocating, community, emotional s...",NaN,West,14.05
5,Siegesaüle,"52.4919769,13.3646303",Magazine,1984,Gays,"Narrating lives, celebrating, meeting, love, s...",NaN,West,14.05
6,Begine,"52.4967592,13.3618488",Squat,1986,Lesbians,"Alternative way of life, alternative environme...",NaN,West,14.05
7,Blocksberg,"52.4932746887207,13.36770248413086",Bar,1975,Lesbians,"sex, dating, togetherness, knowledge sharing, ...",NaN,West,14.05
8,Kaiserhof,"52.4965733,13.3653082",Party,1920,"Lebians, trans persons, gays","sex, dating, togetherness, alternative tempora...",NaN,West,14.05
9,Isherwood,"52.4977986,13.3513834",Individual (writer),1929-1933,Gays,"Narrating lives, creation, art, long-term tran...",NaN,West,14.05


## Carte

In [10]:
coords=df["Adresse"].tolist()
places=df["Place"].tolist()
typologies=df["Type"].tolist()

# coordonnées 
cleaned_coords = [(lat, lon) for coord in coords if isinstance(coord, str) and ',' in coord for lat, lon in [coord.split(',')]]
coords_with_places = [(coord[0], coord[1], place, typology) for coord, place, typology in zip(cleaned_coords, places, typologies)]

# Create a map centered at the first coordinate
map_center = (float(coords_with_places[0][0]), float(coords_with_places[0][1]))
my_map = folium.Map(location=map_center, zoom_start=13)

# Add markers for each coordinate
for loc in coords_with_places:
    lat, lon, place, typologies = loc
    folium.Marker(
        location=(float(lat), float(lon)),
        tooltip=f"""Place: {place} <br>
        Type : {typologies}"""
    ).add_to(my_map)

# Save the map to an HTML file
my_map
